Hosted copy:
* https://gist.github.com/dwt/cf97661734c315731fdc647319ae7d03
* https://tinyurl.com/pyexpect-ipynb

# How to write the most readable unit test code

* Clear definition of actual and expected value
* Error messages that map clearly back to the code you write
* Expressive assertions/ matchers

In [1]:
import unittest

def run_test(a_test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(a_test_class)
    runner = unittest.TextTestRunner(verbosity=0)
    runner.run(suite)

In [2]:
import ipytest
import ipytest.magics

ipytest.config.rewrite_asserts = True

__file__ = "pyexpect.ipynb"

## Expected, Actual?

In [4]:
unsorted = [3,2,1,5,8]

class ExpectedActualConfusionTest(unittest.TestCase):
    
    def test_equals(self):
        self.assertEqual(sorted(unsorted), [1,2,3,5])
        # actual, expected?

    def test_equals_reversed(self):
        self.assertEqual([1,2,3,5], sorted(unsorted))
        # expected, actual?

run_test(ExpectedActualConfusionTest)

FAIL: test_equals (__main__.ExpectedActualConfusionTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-a19c8d7db4a9>", line 6, in test_equals
    self.assertEqual(sorted(unsorted), [1,2,3,5])
AssertionError: Lists differ: [1, 2, 3, 5, 8] != [1, 2, 3, 5]

First list contains 1 additional elements.
First extra element 4:
8

- [1, 2, 3, 5, 8]
?            ---

+ [1, 2, 3, 5]

FAIL: test_equals_reversed (__main__.ExpectedActualConfusionTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-a19c8d7db4a9>", line 10, in test_equals_reversed
    self.assertEqual([1,2,3,5], sorted(unsorted))
AssertionError: Lists differ: [1, 2, 3, 5] != [1, 2, 3, 5, 8]

Second list contains 1 additional elements.
First extra element 4:
8

- [1, 2, 3, 5]
+ [1, 2, 3, 5, 8]
?            +++


----------------------------------------------------------

## Error messages

In [3]:
foo, bar = 3, 4

assert foo == bar

AssertionError: assert 3 == 4

In [9]:
class ErrorMessageTest(unittest.TestCase):
    def test_equals(self): self.assertEqual(1,2)
    def test_in(self): self.assertIn('ab', 'bbb')
    def test_is_not(self): self.assertIsNot(1, 1)

run_test(ErrorMessageTest)

FAIL: test_equals (__main__.ErrorMessageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-9-e3450c60df18>", line 2, in test_equals
    def test_equals(self): self.assertEqual(1,2)
AssertionError: 1 != 2

FAIL: test_in (__main__.ErrorMessageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-9-e3450c60df18>", line 3, in test_in
    def test_in(self): self.assertIn('ab', 'bbb')
AssertionError: 'ab' not found in 'bbb'

FAIL: test_is_not (__main__.ErrorMessageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-9-e3450c60df18>", line 4, in test_is_not
    def test_is_not(self): self.assertIsNot(1, 1)
AssertionError: unexpectedly identical: 1

----------------------------------------------------------------------
Ran 3 tests in 0.000s

FAILED (f

In [4]:
%%run_pytest[clean]

import numpy as np
obj = np.int8(3)
def test_bad_error_message():
    assert isinstance(obj, int)

==================================================================== test session starts ====================================================================
platform darwin -- Python 3.7.2, pytest-4.2.0, py-1.7.0, pluggy-0.8.1
rootdir: /Users/dwt/Code/Projekte/pyexpect, inifile:
collected 1 item

pyexpect.py F                                                                                                                                         [100%]

========================================================================= FAILURES ==========================================================================
__________________________________________________________________ test_bad_error_message ___________________________________________________________________

    def test_bad_error_message():
>       assert isinstance(obj, int)
E       assert False
E        +  where False = isinstance(3, int)

<ipython-input-4-623ff194f1da>:5: AssertionError
========================================

In [2]:
from pyexpect import expect

In [6]:
%%run_pytest[clean]

foo, bar = 3, 4

def test_equals():
    expect(foo).to_equal(bar) # many variant spellings, see source

def test_equals_shorthand():
    expect(foo) == bar # if you like that better

==================================================================== test session starts ====================================================================
platform darwin -- Python 3.7.2, pytest-4.2.0, py-1.7.0, pluggy-0.8.1
rootdir: /Users/dwt/Code/Projekte/pyexpect, inifile:
collected 2 items

pyexpect.py FF                                                                                                                                        [100%]

========================================================================= FAILURES ==========================================================================
________________________________________________________________________ test_equals ________________________________________________________________________

    def test_equals():
>       expect(foo).to_equal(bar) # many variant spellings, see source
E       AssertionError: Expect 3 to equal 4

<ipython-input-6-3fb134c91d73>:5: AssertionError
_____________________________________

## Concise Matchers

In [10]:
expect(23).to_equal(23)
expect(object()).is_trueish()
expect(1).is_true()

AssertionError: Expect 1 to be True

In [11]:
expect(23).not_to_equal(23) # not_* variant always autogenerated

AssertionError: Expect 23 not to equal 23

In [12]:
something = dict(id=23, name='fnord', email='fnord@fnord.fnord')
expect(something).has_subdict(id=23, name='fnord', email='fnord@example.com')

AssertionError: Expect {'id': 23, 'name': 'fnord', 'email': 'fnord@fnord.fnord'}

to contain dict {'id': 23, 'name': 'fnord', 'email': 'fnord@example.com'}

In [13]:
class Foo:
    id = 12
    name = 'fnord'
    email = 'fnord@fnord.fnord'
expect(Foo()).has_attributes(id=12, name='fnord', email='fnord@example.com')

AssertionError: Expect <__main__.Foo object at 0x11bf496d8> to have attributes {'id': 12, 'name': 'fnord', 'email': 'fnord@example.com'}, 
	but has {'id': 12, 'name': 'fnord', 'email': 'fnord@fnord.fnord'}

In [14]:
expect([1,2,3]).has_sub_sequence(1,3)

AssertionError: Expect [1, 2, 3] to contain sequence (1, 3)

In [15]:
def a_function():
    assert False, 'whatever'
expect(a_function).to_raise(AssertionError, r'wh[ia]chever')

AssertionError: Expect <function a_function at 0x11bf581e0> to raise AssertionError with message matching:
	r'wh[ia]chever'
but it raised:
	AssertionError('whatever\nassert False')

In [16]:
expect([23, 42]).has_len(3)

AssertionError: Expect [23, 42] to have length 3, but found length 2

In [17]:
expect(3.14136).close_to(3.3, max_delta=.1)

AssertionError: Expect 3.14136 to be close to 3.3 with max delta 0.1

In [18]:
expect(3.14136).between(3.2, 3.3)

AssertionError: Expect 3.14136 to be between 3.2 and 3.3

In [19]:
expect('Martin Häcker').matches(r'Nitram')

AssertionError: Expect 'Martin Häcker' to be matched by regex r'Nitram'

In [20]:
expect(3) < 4
expect(3) == 3
expect(5) >= 3
# ...

# Takeaway

* much readable
* concisely writeable (aliasses)
* single namespace for all assertions (completion)
* can use everywhere (jupyter notebooks!)
* And much more -> read the source!